In [1]:
# import all modules
# Third-party
import numpy as np
import torch

# Local files
import utilities
import train_NN as train
import neural_network as net

# Choose a device to run on - either CPU or GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# Choose a ground truth PDE to analyze
a = 4 # PDE parameter

# True PDE - Network will satify this PDE at the end of training
def pde_fn(x, a):
    # PDE: u_xx = -(pi*a)^2.sin(pi*a*x)
    u_xx = -(np.pi * a)**2 * np.sin(np.pi * a * x)
    return u_xx

# True solution - Network will try to learn this
def u(x, a):
    return np.sin(np.pi * a * x)

In [3]:
# Create a dataset by sampling points from the PDE's domain
    #1. For the boundaries - subscript u
    #2. Inside the domain = residual - subscript r
# Here, 1D domain, so boundary is just two points! 
bc1_coords = np.array([[0.0], 
                       [0.0]]) 

bc2_coords = np.array([[1.0], 
                       [1.0]])

dom_coords = np.array([[0.0],
                       [1.0]])

no_of_data_samples  = 100
X_bc1 = dom_coords[0, 0] * np.ones((no_of_data_samples // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((no_of_data_samples // 2, 1))

X_u = np.vstack([X_bc1, X_bc2]) # data for BC
Y_u = u(X_u, a) # y data for BC

X_r = np.linspace(dom_coords[0, 0],
                  dom_coords[1, 0], no_of_data_samples)[:, None] # data for residual
Y_r = pde_fn(X_r, a) # y data for residual

# values for normalizing teh datasets
# Normalize the inputs - X_u and X_r - (n_data_points, dimension)
mean = X_r.mean(axis=0)  # (1, dim) values
std = X_r.std(axis=0)  # (1, dim) values   

In [5]:
# Define model and a random seed
seed = 1
max_iterations = 10
model = net.PINN(no_of_neurons=50, no_of_h_layers=2,
                 mean= mean.item(), std =std.item())
model.to(device)

# Train the model -> default SGD with full batch and exponential weight decay
save_data_location = './data_ntk/'
details = train.train_nn_model(model, train_data=(X_u, Y_u, X_r, Y_r),
                            no_iterations=max_iterations, device=device,
                            save_data_location=save_data_location,
                            save_data_frequency=1)
details.keys()

dict_keys(['Loss', 'L_b', 'L_r'])

In [6]:
# Calculate the eigenvalues
import pickle
# load the jacobian matrices from hard disk
J_u, J_r = pickle.load(open("{}/Ju_Jr_{}.p".format(save_data_location,0),
                            'rb'))
eigs = utilities.calculate_eigenvalues_from_j(J_u, J_r)

In [7]:
eigs

array([ 1.83495878e+03,  6.99428733e+02,  2.46027731e+02,  7.98481723e+01,
        6.24142536e+01,  4.82314683e+01,  2.82193289e+01,  2.02984192e+01,
        1.44468807e+01,  1.12866252e+01,  1.11835641e+01,  8.85459490e+00,
        7.74375464e+00,  7.28499994e+00,  6.05934375e+00,  5.17363676e+00,
        5.03138231e+00,  4.47984371e+00,  3.94060239e+00,  3.52129218e+00,
        3.17103983e+00,  3.06609110e+00,  2.92966404e+00,  2.79747043e+00,
        2.54581318e+00,  2.51543161e+00,  2.24219096e+00,  1.99551531e+00,
        1.78071415e+00,  1.65314077e+00,  1.59556050e+00,  1.57105631e+00,
        1.43612036e+00,  1.30920007e+00,  1.29983099e+00,  1.18365836e+00,
        1.08467656e+00,  1.00052359e+00,  9.41858922e-01,  9.29297467e-01,
        8.69570826e-01,  8.19515809e-01,  7.84421988e-01,  7.41826073e-01,
        7.30908562e-01,  7.28533554e-01,  6.76516362e-01,  6.45997746e-01,
        6.03996906e-01,  5.63073766e-01,  5.50644391e-01,  5.46770148e-01,
        5.25022029e-01,  